## Embeddings Trainer

##### 1. Load env variables

In [ ]:
# pip install python-dotenv
from dotenv import load_dotenv
import os

# Carga las variables de entorno del archivo .env
load_dotenv()

# Accede a las variables de entorno
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

##### 2. Import jsonl and load rules

In [7]:
import json

rules = []

# Abre el archivo jsonl
with open('./data/rules_and_regulations.jsonl', 'r', encoding='latin1') as f:
    # Lee cada línea del archivo
    for line in f:
        # Carga la línea como JSON
        data = json.loads(line)

        # Agrega el texto de la línea a la lista de reglas
        rules.append(data["rule"])

        # Imprime el contenido del JSON
        # print(data)

##### 3. Generate Embeddings

In [25]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
   input=rules,
   model="text-embedding-ada-002"
)

print(response)

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.009604336693882942, -0.0021041417494416237, 0.025173069909214973, -0.03228691965341568, -0.024994265288114548, -0.012082051485776901, -0.01493014581501484, 0.0068584163673222065, -0.00989808700978756, -0.05113798379898071, -0.00013550001312978566, 0.031290724873542786, -0.01890215277671814, -0.005993132013827562, -0.013499712571501732, 0.02527524344623089, 0.02181410603225231, -0.036118436604738235, 0.01890215277671814, -0.019132044166326523, -0.009802298620343208, 0.01772715337574482, -0.03284887596964836, -0.004067794419825077, -0.02615649253129959, -0.005846256855875254, 0.020498618483543396, -0.00963626615703106, 0.008301621302962303, -0.015479329973459244, 0.03647604584693909, -0.02774018608033657, -0.009483005851507187, -0.007854610681533813, 5.9518231864785776e-05, 0.00667322613298893, 0.0016427630325779319, -0.010466428473591805, 0.016283949837088585, -0.008014257065951824, 0.017305687069892883, -0.002659711753949523, -0.005

##### 4. Save Embeddings to csv

In [46]:
import pandas as pd

# Crea una lista para almacenar los datos
data = []

# Recorre cada respuesta en la lista de respuestas
for i, _ in enumerate(response.data):
    # Añade el id, la regla y el embedding a la lista de datos
    data.append([i, rules[i], response.data[i].embedding])

# Crea un DataFrame de pandas con los datos
df = pd.DataFrame(data, columns=['id', 'rule', 'embeddings'])

# get size of response.data
print(len(response.data))

29


In [45]:
# Guarda el DataFrame en un archivo CSV
df.to_csv('data/embeddings.csv', index=False, encoding='latin1')

##### 5. Testing comparation

In [38]:
from scipy.spatial.distance import cosine

def cosine_similarity(vec1, vec2):
    # La función 'cosine' de scipy calcula la distancia del coseno, que es 1 menos la similitud del coseno
    return 1 - cosine(vec1, vec2)

result = cosine_similarity(response.data[0].embedding, response.data[1].embedding)